In [4]:
from six.moves import cPickle
import cv2
import numpy as np
from scipy.signal import convolve2d
import theano
import gym
from gym import wrappers
import theano
import theano.tensor as T
import lasagne

In [17]:
from lasagne.layers.dnn import Conv3DDNNLayer
from lasagne.layers.dnn import Pool3DDNNLayer

In [2]:
def _process_frame42(frame):
    frame = frame[34:34+160, :160]
    # Resize by half, then down to 42x42 (essentially mipmapping). If
    # we resize directly we lose pixels that, when mapped to 42x42,
    # aren't close enough to the pixel boundary.
    frame = cv2.resize(frame, (80, 80))
    frame = cv2.resize(frame, (42, 42))
    frame = frame.mean(2)
    frame = frame.astype(np.float32)
    frame *= (1.0 / 255.0)
    frame = np.reshape(frame, [42, 42, 1])
    return frame

In [5]:
env = gym.make("Skiing-v0")

[2017-02-06 16:41:39,038] Making new env: Skiing-v0


In [62]:
dtensor = T.TensorType('float32' ,(False,)*5)

In [72]:
def CreateNeuralNetwork(num_filters=10, filter_size=5, width=10):
    input_var = dtensor('inputs')
    target_var = dtensor('targets')
    network = lasagne.layers.InputLayer(shape=(None, 1, 4, 42, 42), input_var=input_var)
    network = Conv3DDNNLayer(network, num_filters, (2, 5, 5))
    network = lasagne.layers.DenseLayer(network, width, nonlinearity=lasagne.nonlinearities.leaky_rectify)
    features_prediction = lasagne.layers.get_output(network, deterministic=True)
    
    features_prediction_fn = theano.function([input_var], features_prediction)
    return features_prediction_fn

In [68]:
np.array(observation_frame).shape

(4, 42, 42)

In [73]:
pred_fn = CreateNeuralNetwork()

In [74]:
pred_fn(np.array([[observation_frame]]).astype('float32'))

array([[-0.00973749, -0.00627535,  0.37364206,  0.68689555,  0.10219072,
         0.03156561,  0.00909974, -0.00139106, -0.00292248,  0.22306852]], dtype=float32)

In [9]:
np.array(observation_frame).shape

(4, 42, 42)

In [7]:
observation = env.reset()
done = False
iteration, all_reward = 0, 0
observation_frame = []
while not done:
    #env.render()
    observation, reward, done, info = env.step(env.action_space.sample())
    observation_frame += [_process_frame42(observation)[:, :, 0]]
    if len(observation_frame) > 4:
        observation_frame.pop(0)
    print(len(observation_frame))
    all_reward += reward
    # if iteration % 500 == 0:
    # print(str(iteration) + "... ")

    if all_reward < -1000 or iteration >= 8000:
            break

    iteration += 1

print "Reward: ", all_reward

1
2
3
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
Reward:  -1002.0
